In [845]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 11
year = 2022

data = get_data(day=day, year=year)

# submit(cube.sum(), part="a", day=day, year=year)

print(data)

Monkey 0:
  Starting items: 71, 56, 50, 73
  Operation: new = old * 11
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 7

Monkey 1:
  Starting items: 70, 89, 82
  Operation: new = old + 1
  Test: divisible by 7
    If true: throw to monkey 3
    If false: throw to monkey 6

Monkey 2:
  Starting items: 52, 95
  Operation: new = old * old
  Test: divisible by 3
    If true: throw to monkey 5
    If false: throw to monkey 4

Monkey 3:
  Starting items: 94, 64, 69, 87, 70
  Operation: new = old + 2
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 6

Monkey 4:
  Starting items: 98, 72, 98, 53, 97, 51
  Operation: new = old + 6
  Test: divisible by 5
    If true: throw to monkey 0
    If false: throw to monkey 5

Monkey 5:
  Starting items: 79
  Operation: new = old + 7
  Test: divisible by 2
    If true: throw to monkey 7
    If false: throw to monkey 0

Monkey 6:
  Starting items: 77, 55, 63, 93, 66, 90, 88, 71
  Oper

In [765]:
data = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

96578

In [848]:
blocks = [[l.split(': ')[1] for l in block.split("\n")[1:]] for block in data.split("\n\n")]

monkeys = [
    ([int(i) for i in start.split(", ")], 
    op_str.split(' = ')[1], 
    [int(t.split()[-1]) for t in test_args]) 
 for start,op_str,*test_args in blocks]
    
op = lambda op_str,old: eval(op_str) // 3
test = lambda div_by,t,f,val: f if val%div_by else t

inspections = [0]*len(monkeys)

for round in range(20):
    for i, (objects,op_str,test_args) in enumerate(monkeys):
        for obj in objects:
            obj = op(op_str, obj)
            monkeys[test(*test_args, obj)][0].append(obj)
        inspections[i] += len(objects)
        objects[:] = []
            
# for i,m in enumerate(monkeys):
#     print(i, m[0])
    
m1, m2 = sorted(inspections)[-2:]
print(m1*m2)

66802


In [855]:
import numpy as np

blocks = [[l.split(': ')[1] for l in block.split("\n")[1:]] for block in data.split("\n\n")]

tot_rounds = 20
extra_rule = ' // 3'

monkeys = [
    ([int(i) for i in start.split(", ")], 
    op_str.split(' = ')[1], 
    [int(t.split()[-1]) for t in test_args]) 
 for start,op_str,*test_args in blocks]
    
mod_val = np.prod([m[2][0] for m in monkeys])

op = lambda op_str,old: eval(f"({op_str}){extra_rule} % {mod_val}")
test = lambda div_by,t,f,val: f if val%div_by else t

inspections = [0]*len(monkeys)

for round in range(tot_rounds):
    for i, (objects,op_str,test_args) in enumerate(monkeys):
        for obj in objects:
            obj = op(op_str, obj)
            monkeys[test(*test_args, obj)][0].append(obj)
        inspections[i] += len(objects)
        objects[:] = []
            
# for i,m in enumerate(monkeys):
#     print(i, m[0])

m1, m2 = sorted(inspections)[-2:]
print(m1*m2)

66802


In [810]:
for i, (objects,op_str,test_args) in enumerate(monkeys):
    for obj in objects:
        obj = op(op_str, obj)
        monkeys[test(*test_args, obj)][0].append(obj)
    inspections[i] += len(objects)
    objects[:] = []
            
for i,m in enumerate(monkeys):
    print(i, m[0])

for i,m in enumerate(monkeys):
    print(i)
    for x in m[0]:
        print([(x%d) == 0 for d in (23,19,13,17)])

# m1, m2 = sorted(inspections)[-2:]
# print(m1*m2)

0 [11177067658820655620059127953910208049, 69091264919368945253705172586649917316629, 437765284184743071223940162023881344230680, 843565370584631059762690886495521558288758, 580080386464772677915717618845677139721993078, 173972242326876201666612793205724895418812113211643584759]
1 [1368353957993036415454840837978342947595408861409253, 6726782135096948773522899814723, 329330942975254545383024023041826496051996868563568240125839626676, 7223640457929552026175308189194558783010275257855913447]
2 []
3 []
0
[False, False, False, False]
[False, False, False, False]
[False, False, False, False]
[False, False, False, False]
[False, False, True, False]
[False, False, False, False]
1
[False, False, False, False]
[False, False, False, False]
[False, False, False, False]
[False, False, False, False]
2
3


In [799]:
for i,m in enumerate(monkeys):
    print(i)
    for x in m[0]:
        print([(x%d) == 0 for d in (23,19,13,17)])


0
[False, False, False, False]
[False, False, False, False]
1
[False, False, False, False]
[False, False, False, False]
[False, False, False, False]
[False, False, True, False]
[False, False, True, False]
[False, False, False, False]
[False, False, False, False]
[False, False, False, False]
2
3


['D', 'C']